#### Experiment: 

Evaluate pruning by magnitude weighted by coactivations (more thorough evaluation), compare it to baseline (SET).

#### Motivation.

Check if results are consistently above baseline.

#### Conclusion

- No significant difference between both models
- No support for early stopping

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import glob
import tabulate
import pprint
import click
import numpy as np
import pandas as pd
from ray.tune.commands import *
from nupic.research.frameworks.dynamic_sparse.common.browser import *

## Load and check data

In [8]:
exps = ['improved_magpruning_eval1', ]
paths = [os.path.expanduser("~/nta/results/{}".format(e)) for e in exps]
df = load_many(paths)

In [9]:
df.head(5)

,Experiment Name,train_acc_max,train_acc_max_epoch,train_acc_min,train_acc_min_epoch,train_acc_median,train_acc_last,val_acc_max,val_acc_max_epoch,val_acc_min,...,momentum,network,num_classes,on_perc,optim_alg,pruning_early_stop,test_noise,use_kwinners,weight_decay,weight_prune_perc
0,"0_model=DSNNWeightedMag,on_perc=0.2,pruning_ea...",0.999483,93,0.922517,0,0.998808,0.999083,0.9802,35,0.9597,...,0.9,MLPHeb,10,0.2,SGD,0,False,False,0.0001,NaN
1,"1_model=DSNNMixedHeb,on_perc=0.2,pruning_early...",0.999433,87,0.926283,0,0.998683,0.999333,0.9798,35,0.9603,...,0.9,MLPHeb,10,0.2,SGD,0,False,False,0.0001,NaN
2,"2_model=DSNNWeightedMag,on_perc=0.1,pruning_ea...",0.993517,92,0.908733,0,0.990750,0.993150,0.9733,92,0.9506,...,0.9,MLPHeb,10,0.1,SGD,0,False,False,0.0001,NaN
3,"3_model=DSNNMixedHeb,on_perc=0.1,pruning_early...",0.993217,94,0.905483,0,0.990275,0.993017,0.9725,38,0.9502,...,0.9,MLPHeb,10,0.1,SGD,0,False,False,0.0001,NaN
4,"4_model=DSNNWeightedMag,on_perc=0.2,pruning_ea...",0.999400,75,0.927883,0,0.998633,0.999050,0.9818,44,0.9640,...,0.9,MLPHeb,10,0.2,SGD,1,False,False,0.0001,NaN


In [10]:
# replace hebbian prine
df['hebbian_prune_perc'] = df['hebbian_prune_perc'].replace(np.nan, 0.0, regex=True)
df['weight_prune_perc'] = df['weight_prune_perc'].replace(np.nan, 0.0, regex=True)

In [11]:
df.columns

Index(['Experiment Name', 'train_acc_max', 'train_acc_max_epoch',
       'train_acc_min', 'train_acc_min_epoch', 'train_acc_median',
       'train_acc_last', 'val_acc_max', 'val_acc_max_epoch', 'val_acc_min',
       'val_acc_min_epoch', 'val_acc_median', 'val_acc_last', 'epochs',
       'experiment_file_name', 'trial_time', 'mean_epoch_time', 'batch_norm',
       'data_dir', 'dataset_name', 'debug_sparse', 'debug_weights', 'device',
       'hebbian_grow', 'hebbian_prune_perc', 'hidden_sizes', 'input_size',
       'learning_rate', 'lr_gamma', 'lr_milestones', 'lr_scheduler', 'model',
       'momentum', 'network', 'num_classes', 'on_perc', 'optim_alg',
       'pruning_early_stop', 'test_noise', 'use_kwinners', 'weight_decay',
       'weight_prune_perc'],
      dtype='object')

In [12]:
df.shape

(288, 42)

In [13]:
df.iloc[1]

Experiment Name         1_model=DSNNMixedHeb,on_perc=0.2,pruning_early...
train_acc_max                                                    0.999433
train_acc_max_epoch                                                    87
train_acc_min                                                    0.926283
train_acc_min_epoch                                                     0
train_acc_median                                                 0.998683
train_acc_last                                                   0.999333
val_acc_max                                                        0.9798
val_acc_max_epoch                                                      35
val_acc_min                                                        0.9603
val_acc_min_epoch                                                       0
val_acc_median                                                     0.9784
val_acc_last                                                       0.9789
epochs                                

In [14]:
df.groupby('model')['model'].count()

model
DSNNMixedHeb       144
DSNNWeightedMag    144
Name: model, dtype: int64

 ## Analysis

Experiment Details

In [15]:
# Did any  trials failed?
df[df["epochs"]<30]["epochs"].count()

0

In [16]:
# Removing failed or incomplete trials
df_origin = df.copy()
df = df_origin[df_origin["epochs"]>=30]
df.shape

(288, 42)

In [17]:
# which ones failed?
# failed, or still ongoing?
df_origin['failed'] = df_origin["epochs"]<30
df_origin[df_origin['failed']]['epochs']

Series([], Name: epochs, dtype: int64)

In [18]:
# helper functions
def mean_and_std(s):
    return "{:.3f} ± {:.3f}".format(s.mean(), s.std())

def round_mean(s):
    return "{:.0f}".format(round(s.mean()))

stats = ['min', 'max', 'mean', 'std']

def agg(columns, filter=None, round=3):
    if filter is None:
        return (df.groupby(columns)
             .agg({'val_acc_max_epoch': round_mean,
                   'val_acc_max': stats,                
                   'model': ['count']})).round(round)
    else:
        return (df[filter].groupby(columns)
             .agg({'val_acc_max_epoch': round_mean,
                   'val_acc_max': stats,                
                   'model': ['count']})).round(round)


##### Does improved weight pruning outperforms regular SET

In [20]:
agg(['model'])

val_acc_max_epoch val_acc_max                      model
                       round_mean         min    max   mean    std count
model                                                                   
DSNNMixedHeb                   49       0.972  0.986  0.982  0.003   144
DSNNWeightedMag                48       0.973  0.986  0.981  0.003   144

In [22]:
agg(['on_perc', 'model'])

val_acc_max_epoch val_acc_max                       \
                               round_mean         min    max   mean    std   
on_perc model                                                                
0.1     DSNNMixedHeb                   46       0.972  0.984  0.980  0.003   
        DSNNWeightedMag                47       0.973  0.983  0.980  0.003   
0.2     DSNNMixedHeb                   53       0.980  0.986  0.983  0.002   
        DSNNWeightedMag                49       0.979  0.986  0.983  0.001   

                        model  
                        count  
on_perc model                  
0.1     DSNNMixedHeb       72  
        DSNNWeightedMag    72  
0.2     DSNNMixedHeb       72  
        DSNNWeightedMag    72

In [24]:
agg(['weight_prune_perc', 'model'])

val_acc_max_epoch val_acc_max                \
                                         round_mean         min    max   mean   
weight_prune_perc model                                                         
0.0               DSNNMixedHeb                   38       0.972  0.982  0.978   
                  DSNNWeightedMag                42       0.973  0.983  0.978   
0.1               DSNNMixedHeb                   48       0.980  0.986  0.982   
                  DSNNWeightedMag                49       0.979  0.985  0.983   
0.2               DSNNMixedHeb                   48       0.979  0.985  0.982   
                  DSNNWeightedMag                51       0.980  0.985  0.982   
0.3               DSNNMixedHeb                   55       0.980  0.985  0.983   
                  DSNNWeightedMag                52       0.980  0.986  0.983   
0.4               DSNNMixedHeb                   53       0.981  0.985  0.983   
                  DSNNWeightedMag                52       0.979  0.985  0.982   
0.5               DSNNMixedHeb                   53       0.979  0.985  0.982   
                  DSNNWeightedMag                43       0.978  0.984  0.981   

                                         model  
                                     std count  
weight_prune_perc model                         
0.0               DSNNMixedHeb     0.003    24  
                  DSNNWeightedMag  0.003    24  
0.1               DSNNMixedHeb     0.002    24  
                  DSNNWeightedMag  0.002    24  
0.2               DSNNMixedHeb     0.002    24  
                  DSNNWeightedMag  0.002    24  
0.3               DSNNMixedHeb     0.001    24  
                  DSNNWeightedMag  0.002    24  
0.4               DSNNMixedHeb     0.002    24  
                  DSNNWeightedMag  0.002    24  
0.5               DSNNMixedHeb     0.002    24  
                  DSNNWeightedMag  0.002    24

In [ ]:
agg(['on_perc', 'pruning_early_stop', 'model'])

In [28]:
agg(['on_perc', 'pruning_early_stop', 'model'])

val_acc_max_epoch val_acc_max  \
                                                  round_mean         min   
on_perc pruning_early_stop model                                           
0.1     0                  DSNNMixedHeb                   47       0.972   
                           DSNNWeightedMag                52       0.973   
        1                  DSNNMixedHeb                   43       0.973   
                           DSNNWeightedMag                54       0.974   
        2                  DSNNMixedHeb                   50       0.974   
                           DSNNWeightedMag                38       0.973   
        3                  DSNNMixedHeb                   43       0.974   
                           DSNNWeightedMag                46       0.975   
0.2     0                  DSNNMixedHeb                   46       0.980   
                           DSNNWeightedMag                48       0.980   
        1                  DSNNMixedHeb                   54       0.980   
                           DSNNWeightedMag                49       0.979   
        2                  DSNNMixedHeb                   53       0.980   
                           DSNNWeightedMag                49       0.979   
        3                  DSNNMixedHeb                   57       0.980   
                           DSNNWeightedMag                52       0.981   

                                                                model  
                                              max   mean    std count  
on_perc pruning_early_stop model                                       
0.1     0                  DSNNMixedHeb     0.983  0.980  0.003    18  
                           DSNNWeightedMag  0.982  0.979  0.003    18  
        1                  DSNNMixedHeb     0.983  0.980  0.003    18  
                           DSNNWeightedMag  0.983  0.980  0.003    18  
        2                  DSNNMixedHeb     0.984  0.980  0.003    18  
                           DSNNWeightedMag  0.983  0.980  0.003    18  
        3                  DSNNMixedHeb     0.982  0.980  0.003    18  
                           DSNNWeightedMag  0.983  0.980  0.002    18  
0.2     0                  DSNNMixedHeb     0.985  0.983  0.002    18  
                           DSNNWeightedMag  0.985  0.983  0.001    18  
        1                  DSNNMixedHeb     0.985  0.984  0.002    18  
                           DSNNWeightedMag  0.986  0.983  0.001    18  
        2                  DSNNMixedHeb     0.986  0.983  0.002    18  
                           DSNNWeightedMag  0.985  0.983  0.002    18  
        3                  DSNNMixedHeb     0.985  0.984  0.001    18  
                           DSNNWeightedMag  0.985  0.983  0.001    18

* No significant difference between the two approaches

##### What is the impact of early stopping?

In [29]:
agg(['pruning_early_stop'])

val_acc_max_epoch val_acc_max                      model
                          round_mean         min    max   mean    std count
pruning_early_stop                                                         
0                                 48       0.972  0.985  0.982  0.003    72
1                                 50       0.973  0.986  0.982  0.003    72
2                                 48       0.973  0.986  0.981  0.003    72
3                                 49       0.974  0.985  0.981  0.003    72

In [32]:
agg(['model', 'pruning_early_stop'])

val_acc_max_epoch val_acc_max         \
                                          round_mean         min    max   
model           pruning_early_stop                                        
DSNNMixedHeb    0                                 46       0.972  0.985   
                1                                 49       0.973  0.985   
                2                                 52       0.974  0.986   
                3                                 50       0.974  0.985   
DSNNWeightedMag 0                                 50       0.973  0.985   
                1                                 51       0.974  0.986   
                2                                 44       0.973  0.985   
                3                                 49       0.975  0.985   

                                                 model  
                                     mean    std count  
model           pruning_early_stop                      
DSNNMixedHeb    0                   0.982  0.003    36  
                1                   0.982  0.003    36  
                2                   0.982  0.003    36  
                3                   0.982  0.003    36  
DSNNWeightedMag 0                   0.981  0.003    36  
                1                   0.982  0.003    36  
                2                   0.981  0.003    36  
                3                   0.981  0.002    36

In [30]:
agg(['on_perc', 'pruning_early_stop'])

val_acc_max_epoch val_acc_max                       \
                                  round_mean         min    max   mean    std   
on_perc pruning_early_stop                                                      
0.1     0                                 49       0.972  0.983  0.980  0.003   
        1                                 48       0.973  0.983  0.980  0.003   
        2                                 44       0.973  0.984  0.980  0.003   
        3                                 44       0.974  0.983  0.980  0.002   
0.2     0                                 47       0.980  0.985  0.983  0.001   
        1                                 51       0.979  0.986  0.983  0.001   
        2                                 51       0.979  0.986  0.983  0.002   
        3                                 54       0.980  0.985  0.983  0.001   

                           model  
                           count  
on_perc pruning_early_stop        
0.1     0                     36  
        1                     36  
        2                     36  
        3                     36  
0.2     0                     36  
        1                     36  
        2                     36  
        3                     36

* Results are not strong enough. But it is has a slight increase in acc (0.1%), not greater than the standard deviation, when pruning stops at the first learning rate decay for WeightedMag model and at the second learning rate decay for DSSNMixedHeb models.

